# Validação de processos necessários para criação da pipeline

## Instalação dependências

In [12]:
!pip install -q duckdb pandas boto3


[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## DuckDB
O DuckDB é eficiente e de alto desempenho. Ao trabalhar com arquivos CSV, por exemplo, ele não precisa carregar o arquivo CSV inteiro na memória antes de poder processá-lo. Em vez disso, o DuckDB pode ler e processar dados do arquivo em tempo real. Para ver isso em ação, vamos usar o conjunto de dados de Atrasos e Cancelamentos de Voos de 2015.

## Leitura de arquivo com o DuckDB

In [20]:
import duckdb
duckdb.sql('SELECT * FROM "data/sample.csv"').df()

,Status,Código da avaliação,Código da Unidade,Unidade,Cidade,Estado,Região/Grupo,Código,Nome do checklist,Autor,...,Resultado,Comentário do item,Imagens,Plataforma,Coordenadas Iniciais,Coordenadas Finais,Comentários Finais,Comentários do responsável da unidade,Planos de Ação,Ações
0,Concluído,152622447,2821694,Generic shop,None,None,None,394104,Generic shop,Leidiane Marques,...,"79,56",None,None,iOS,None,None,Auditoria finalizada na unidade,None,None,None
1,Concluído,152622447,2821694,Generic shop,None,None,None,394104,Generic shop,Leidiane Marques,...,"79,56",Ar-condicionado e teto da cozinha com acúmulo ...,https://app.com.br/evaluations,iOS,None,None,Auditoria finalizada,None,None,None


## Mecanismo de processamento incremental

In [10]:
import duckdb
duckdb.sql('SELECT * FROM "data/sample.csv"').df()

,Status,Código da avaliação,Código da Unidade,Unidade,Cidade,Estado,Região/Grupo,Código,Nome do checklist,Autor,...,Resultado,Comentário do item,Imagens,Plataforma,Coordenadas Iniciais,Coordenadas Finais,Comentários Finais,Comentários do responsável da unidade,Planos de Ação,Ações
0,Concluído,152622447,2821694,Generic shop,None,None,None,394104,Generic shop,Leidiane Marques,...,"79,56",None,None,iOS,None,None,Auditoria finalizada na unidade,None,None,None
1,Concluído,152622447,2821694,Generic shop,None,None,None,394104,Generic shop,Leidiane Marques,...,"79,56",Ar-condicionado e teto da cozinha com acúmulo ...,https://app.com.br/evaluations,iOS,None,None,Auditoria finalizada,None,None,None


In [11]:
import duckdb
duckdb.sql('SELECT * FROM "data/sample2.csv"').df()

,Status,Código da avaliação,Código da Unidade,Unidade,Cidade,Estado,Região/Grupo,Código,Nome do checklist,Autor,...,Resultado,Comentário do item,Imagens,Plataforma,Coordenadas Iniciais,Coordenadas Finais,Comentários Finais,Comentários do responsável da unidade,Planos de Ação,Ações
0,Concluído,152622448,2821695,Generic shop,None,None,None,394104,Generic shop,Ruth Andrade,...,"79,56",Necessário realizar higienização,https://app.com.br/evaluations,iOS,None,None,Auditoria finalizada,None,None,None


In [12]:
duckdb.sql('CREATE TABLE raw AS SELECT * FROM "data/sample.csv"')
duckdb.sql('SELECT * FROM raw')

duckdb.sql('''
SELECT * FROM "data/*.csv"
WHERE "Data de sincronização" > (SELECT MAX("Data de sincronização") FROM raw)
''').df()



,Status,Código da avaliação,Código da Unidade,Unidade,Cidade,Estado,Região/Grupo,Código,Nome do checklist,Autor,...,Resultado,Comentário do item,Imagens,Plataforma,Coordenadas Iniciais,Coordenadas Finais,Comentários Finais,Comentários do responsável da unidade,Planos de Ação,Ações
0,Concluído,152622448,2821695,Generic shop,None,None,None,394104,Generic shop,Ruth Andrade,...,"79,56",Necessário realizar higienização,https://app.com.br/evaluations,iOS,None,None,Auditoria finalizada,None,None,None


## Integração do MinIO e DuckDB

### Criação das credenciais

In [14]:
duckdb.sql("""
CREATE OR REPLACE PERSISTENT SECRET my_secret (
TYPE S3,
REGION 'us-east-1',
KEY_ID 'TESTE123',
SECRET 'TESTE123',
ENDPOINT 'localhost:9000',
USE_SSL 'false',
URL_STYLE 'path');
""")


┌─────────┐
│ Success │
│ boolean │
├─────────┤
│ true    │
└─────────┘

### Escrita para o MinIO

In [16]:
duckdb.sql("""
COPY (SELECT
"Código da avaliação" as id,
Unidade as unidade,
Cidade as cidade,
Autor as autor,
FROM 'data/*.csv') TO 's3://hawkeye/checklist.parquet' (FORMAT 'parquet');
""")

### Leitura arquivo do MinIO

In [18]:
duckdb.sql("""
SELECT
id,
unidade,
cidade,
autor,
FROM 's3://hawkeye/checklist.parquet';
""").df()

,id,unidade,cidade,autor
0,152622447,Generic shop,None,Leidiane Marques
1,152622447,Generic shop,None,Leidiane Marques
2,152622448,Generic shop,None,Ruth Andrade


### Leitura e escrita entre arquivos no MinIO

In [19]:
duckdb.sql("""
COPY (SELECT
id as id2,
unidade as unidade2,
cidade as cidade2,
autor as autor2,
FROM 's3://hawkeye/checklist.parquet') TO 's3://hawkeye/checklist2.parquet' (FORMAT 'parquet');
""")

# Seleção de colunas

Conforme acordado com o cliente, para montar os gráficos de dashboards, será necessário utilizar as seguintes colunas:

- Codigo da avaliacao → id
- Unidade → unidade
- Cidade → cidade
- Regiao/Grupo → regiao
- Nome do checklist → nome_checklist
- Autor → autor
- Area → area
- Item → item
- Resposta → resposta
- Imagens → imagens
- Data inicial → data_inicial
- "Data final" → data_final
- "Data de sincronização" → data_sincronizacao
- Resultado → resultado
- Comentarios finais → comentarios_finais


In [7]:
import duckdb
duckdb.sql('''
 SELECT
    "Código da avaliação",
    Unidade,
    Cidade,
    "Região/Grupo",
    "Nome do checklist",
    Autor,
    "Área",
    Item,
    Resposta,
    Imagens,
    "Data inicial",
    "Data final",
    "Data de sincronização",
    Resultado,
    "Comentários finais"
FROM "data/sample.csv"''').df()



,Código da avaliação,Unidade,Cidade,Região/Grupo,Nome do checklist,Autor,Área,Item,Resposta,Imagens,Data inicial,Data final,Data de sincronização,Resultado,Comentários Finais
0,152622447,Generic shop,None,None,Generic shop,Leidiane Marques,COZINHA,Organização,Atingiu,None,2025-07-11 19:16:16,2025-07-11 20:24:17,2025-07-11 21:24:55,"79,56",Auditoria finalizada na unidade
1,152622447,Generic shop,None,None,Generic shop,Leidiane Marques,COZINHA,"Limpeza do ambiente (piso, parede, portas)",Não atingiu,https://app.com.br/evaluations,2025-07-11 19:16:16,2025-07-11 20:24:17,2025-07-11 21:24:55,"79,56",Auditoria finalizada


# Transformações

Durante o processamento dos dados, serão aplicadas as seguintes transformações para padronizar e preparar as informações que alimentarão os dashboards:

- **Remoção de duplicados**: eliminar registros que possuam o mesmo `id`, garantindo consistência e unicidade dos dados, o critério para escolha vai ser o mais recente.
- **Ajuste do campo `resultado`**: converter valores textuais (ex.: `"75,6"`) para formato numérico (`75.6`), permitindo cálculos e comparações estatísticas.
- **Criação do campo `total_fotos`**: extrair automaticamente a quantidade de imagens associadas a cada registro a partir da coluna `imagens`.
- **Padronização de nomes de colunas**: aplicar convenção *snake_case* para todos os campos, assegurando consistência na manipulação e integração dos dados.
- **Remoção de valores nulos**: Troca de valores nulos para `Não definido`.


In [14]:
import duckdb
duckdb.sql('''
WITH raw AS (
     SELECT
    "Código da avaliação" as id,
    Unidade as unidade,
    Cidade as cidade,
    "Região/Grupo" as regiao,
    "Nome do checklist" as nome,
    Autor as autor,
    "Área" as area,
    Item as item,
    Resposta as reposta,
    Imagens as imagens,
    "Data inicial" as data_inicial,
    "Data final" as data_final,
    "Data de sincronização" as data_sincronizacao,
    Resultado AS result,
    "Comentários finais" as final_comments
   FROM  "data/sample.csv"
), cleaned as (
    SELECT
    id,
    unidade,
    COALESCE(cidade, 'Não definido') as cidade,
    COALESCE(regiao, 'Não definido') as regiao,
    nome,
    autor,
    area,
    item,
    reposta,
    COALESCE(len(string_split(imagens, ' ')), 0) as total_fotos,
    CAST(REPLACE(result, ',', '.') AS DOUBLE) AS result,
    data_inicial,
    data_final,
    data_sincronizacao,
    final_comments,
    ROW_NUMBER() OVER (PARTITION BY id ORDER BY data_sincronizacao DESC) as rn
    FROM raw
)
SELECT *
FROM cleaned
WHERE rn=1


    ''').df()



,id,unidade,cidade,regiao,nome,autor,area,item,reposta,total_fotos,result,data_inicial,data_final,data_sincronizacao,final_comments,rn
0,152622448,Generic shop,Não definido,Não definido,Generic shop,Leidiane Marques,COZINHA,"Limpeza do ambiente (piso, parede, portas)",Não atingiu,1,79.56,2025-07-11 19:16:16,2025-07-11 20:24:17,2025-07-11 21:24:55,Auditoria finalizada,1
1,152622447,Generic shop,Não definido,Não definido,Generic shop,Leidiane Marques,COZINHA,Organização,Atingiu,0,79.56,2025-07-11 19:16:16,2025-07-11 20:24:17,2025-07-11 21:24:55,Auditoria finalizada na unidade,1


# Pipeline final

## Criar credencial

In [1]:
import duckdb

In [2]:
duckdb.sql("""
CREATE OR REPLACE PERSISTENT SECRET my_secret (
TYPE S3,
REGION 'us-east-1',
KEY_ID 'TESTE123',
SECRET 'TESTE123',
ENDPOINT 'localhost:9000',
USE_SSL 'false',
URL_STYLE 'path');
""")

┌─────────┐
│ Success │
│ boolean │
├─────────┤
│ true    │
└─────────┘

## Subir arquivo CSV para o MinIO

In [3]:
import boto3

minio_endpoint = "http://localhost:9000"
access_key = "minioadmin"
secret_key = "minioadmin"
bucket_name = "hawkeye"
local_file = "./data/sample.csv"
object_name = "lm/landing/data.csv"

s3_client = boto3.client(
    "s3",
    endpoint_url=minio_endpoint,
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_key,
    region_name="us-east-1"
)

s3_client.upload_file(local_file, bucket_name, object_name)



## Raw

In [4]:
import duckdb


duckdb.sql("""
COPY (
     SELECT
    "Código da avaliação",
    Unidade,
    Cidade,
    "Região/Grupo",
    "Nome do checklist",
    Autor,
    "Área",
    Item,
    Resposta,
    Imagens,
    "Data inicial",
    "Data final",
    "Data de sincronização",
    Resultado,
    "Comentários finais"
    FROM 's3://hawkeye/lm/landing/*.csv'
) TO 's3://hawkeye/lm/raw/checklist.parquet' (FORMAT 'parquet');
""")


## Cleaned

In [10]:
duckdb.sql("""
COPY (
WITH raw AS (
     SELECT
    "Código da avaliação" as id,
    Unidade as unidade,
    Cidade as cidade,
    "Região/Grupo" as regiao,
    "Nome do checklist" as nome,
    Autor as autor,
    "Área" as area,
    Item as item,
    Resposta as reposta,
    Imagens as imagens,
    "Data inicial" as data_inicial,
    "Data final" as data_final,
    "Data de sincronização" as data_sincronizacao,
    Resultado AS result,
    "Comentários finais" as final_comments
   FROM  "s3://hawkeye/lm/raw/checklist.parquet"
), cleaned as (
    SELECT
    id,
    unidade,
    COALESCE(cidade, 'Não definido') as cidade,
    COALESCE(regiao, 'Não definido') as regiao,
    nome,
    autor,
    area,
    item,
    reposta,
    COALESCE(len(string_split(imagens, ' ')), 0) as total_fotos,
    CAST(REPLACE(result, ',', '.') AS DOUBLE) AS result,
    data_inicial,
    data_final,
    data_sincronizacao,
    final_comments,
    FROM raw
)
SELECT DISTINCT id, unidade, cidade, regiao, nome, autor, area, item, reposta, total_fotos, result, data_inicial, data_final, data_sincronizacao, final_comments
FROM cleaned
) TO 's3://hawkeye/lm/cleaned/checklist.parquet' (FORMAT 'parquet');
""")


## Consulta

In [15]:
import duckdb


duckdb.sql("""
SELECT *
FROM 's3://hawkeye/lm/cleaned/checklist.parquet'
""").df()


,id,unidade,cidade,regiao,nome,autor,area,item,reposta,total_fotos,result,data_inicial,data_final,data_sincronizacao,final_comments
0,113921777,Mestre do Pão,Não definido,Não definido,MESTRE DO PÃO COMPENSA,RAHUL RAMCHANDANI,CONFEITARIA,Área precisa de manutenção?,Atingiu parcial,0,51.02,2024-08-13 12:54:20,2024-08-13 14:13:47,2024-08-13 14:16:24,Hoje em visita foram revisados :\n\n*expositor...
1,113919139,SUPERMERCADO COEMA,Não definido,Não definido,COEMA MONTE DAS OLIVEIRAS,Rayane Souza,Peixaria,Ausência de Pragas e/ou Barreiras de controle ...,Não se aplica,0,62.41,2024-08-13 13:03:06,2024-08-13 14:19:33,2024-08-13 18:28:22,\n\nVisita realizada com ênfase nas seguintes ...
2,113913315,COQUEIRO VERDE,Não definido,Não definido,Coqueiro P.10,Karoline Marques,SALÃO,"Limpeza do ambiente (piso, parede, portas)",Atingiu,0,92.58,2024-08-13 12:18:47,2024-08-13 13:20:09,2024-08-13 14:21:16,Auditoria finalizada na unidade \n\nObtive aco...
3,113911556,SUPERMERCADO VENEZA,Não definido,Não definido,VENEZA CENTRO,Alexia Muniz,Área de Vendas Mercearia,Produtos armazenados adequadamente (refrigeraç...,Atingiu,0,68.85,2024-08-13 11:09:11,2024-08-13 12:16:58,2024-08-13 13:17:55,None
4,113903059,REI DO CHURRASCO,Não definido,Não definido,REI DO CHURRASCO CACHOEIRINHA,Karoline Marques,DOCUMENTAÇÃO,A Unidade está com o Alvará de Funcionamento d...,Não se aplica,0,87.27,2024-08-13 10:25:03,2024-08-13 11:36:41,2024-08-13 12:37:15,Auditoria finalizada na unidade \n\nObtive aco...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421256,137872177,SUPERMERCADO GOIANA,Não definido,Não definido,Goiana Centro,SOCORRO LIMA,Manipulação da Padaria,Instalações higienizadas,Não se aplica,0,34.33,2025-03-27 20:26:31,2025-03-27 21:23:29,2025-03-27 23:22:23,None
421257,137852875,SOHO LOUNGE,Não definido,Não definido,SOHO CIDADE CARROS,RAHUL RAMCHANDANI,COZINHA,"Limpeza dos equipamentos (máquinas, bancadas, ...",Não atingiu,1,77.32,2025-03-27 17:23:21,2025-03-27 18:10:34,2025-03-27 18:10:46,Hoje em visita foram pontuados :\n\nSushi \n* ...
421258,137834211,CN SUPERMERCADOS,Não definido,Não definido,CN SUPERMERCADO P10,Fernando Silva,FLV,Higiene dos manipuladores adequadas (local ofe...,Atingiu parcial,1,76.06,2025-03-27 14:29:44,2025-03-27 15:41:17,2025-03-27 16:41:38,Visita finalizada com ênfase em: \n- FLV\n- Pa...
421259,137834211,CN SUPERMERCADOS,Não definido,Não definido,CN SUPERMERCADO P10,Fernando Silva,Manipulação de Frios,"Produtos com aspecto de baixa qualidade, aspec...",Não se aplica,0,76.06,2025-03-27 14:29:44,2025-03-27 15:41:17,2025-03-27 16:41:38,Visita finalizada com ênfase em: \n- FLV\n- Pa...
